In [1]:
import pandas as pd
import numpy as np

In [2]:
document = ["Lawang-Sewu memiliki sistem pendingin air di bawah lantai, dan di atap. Itulah fungsi utama basement #lawangsewu",
            "lawang-sewu berada di ujung Bodjongweg atau yang sekarang dikenal dengan nama Jalan Pemuda",
            "lawang-sewu pernah digunakan sebagai penjara bawah tanah oleh serdadu Jepang.",
            "lawang-sewu merupakan landmark Kota Semarang yang paling dikenal para turis baik lokal maupun asing",
            "Lawang-Sewu adalah salah satu bangunan bersejarah yang dibangun oleh pemerintahan kolonial Belanda, pada 27 Februari 1904.",
            "lawang-sewu dulu menjadi lokasi pertempuran yang hebat antara pemuda Angkatan Muda Kereta Api melawan Kempetai dan Kidobutai Jepang",
            "Masyarakat setempat menyebutnya Lawang-Sewu (Seribu Pintu) dikarenakan bangunan tersebut memiliki pintu yang sangat banyak.",
            "Lawang-Sewu dibangun pada tahun 1904 dan selesai pada tahun 1907. Terletak di bundaran Tugu Muda yang dahulu disebut Wilhelminaplein.",
            "Lawang-Sewu tempo doeloe. Ada kereta api lewat juga di depannya.",
            "di lantai dua terdapat aula besar yg dulu digunakan sebagai ruang pesta. Di sudut aula terpasang wastafel yg sudah ada sejak pertma kali lawang-sewu dibangun."]
len(document)

10

# Preprocessing

In [3]:
# import
from preprocess.normalize import Normalize
from preprocess.tokenize import Tokenize
from preprocess.symspell import SymSpell

In [4]:
# initialize
normalizer = Normalize()
tokenizer = Tokenize()
symspell = SymSpell(max_dictionary_edit_distance=3)
symspell.load_complete_model_from_json('preprocess\data\corpus_complete_model.json', encoding="ISO-8859-1")

Loading dictionary...
Processing dictionary...
Copied 94811 words to master dictionary...
Copied 679534 hashes to master dictionary...


In [5]:
# do process
doc_preprocessed = []

for tweet in document:
    # normalize
    tweet_norm = normalizer.remove_ascii_unicode(tweet)
    tweet_norm = normalizer.remove_rt_fav(tweet_norm)
    tweet_norm = normalizer.lower_text(tweet_norm)
    tweet_norm = normalizer.remove_newline(tweet_norm)
    tweet_norm = normalizer.remove_url(tweet_norm)
    tweet_norm = normalizer.remove_emoticon(tweet_norm)
    tweet_norm = normalizer.remove_hashtag_mention(tweet_norm)
    tweet_norm = normalizer.remove_punctuation(tweet_norm)
    
    # tokenize
    tweet_tok = tokenizer.WordTokenize(tweet_norm, removepunct=True)
    
    # spell correction
    temp = []
    for token in tweet_tok:
        suggestion = symspell.lookup(phrase=token, verbosity=1, max_edit_distance=3)

        # option if there is no suggestion
        if len(suggestion) > 0:
            get_suggestion = str(suggestion[0]).split(':')[0]
            temp.append(get_suggestion)
        else:
            temp.append(token)
    tweet_prepared = ' '.join(temp)
    
    doc_preprocessed.append(tweet_prepared)

In [6]:
for _ in doc_preprocessed:
    print(_)

lawang-sewu memiliki sistem pendingin air di bawah lantai dan di atap itulah fungsi utama basement
lawang-sewu berada di ujung bodjongweg atau yang sekarang dikenal dengan nama jalan pemuda
lawang-sewu pernah digunakan sebagai penjara bawah tanah oleh serdadu jepang
lawang-sewu merupakan landmark kota semarang yang paling dikenal para turis baik lokal maupun asing
lawang-sewu adalah salah satu bangunan bersejarah yang dibangun oleh pemerintahan kolonial belanda pada 27 februari 1904
lawang-sewu dulu menjadi lokasi pertempuran yang hebat antara pemuda angkatan muda kereta api melawan kempetai dan kidobutai jepang
masyarakat setempat menyebutnya lawang-sewu seribu pintu dikarenakan bangunan tersebut memiliki pintu yang sangat banyak
lawang-sewu dibangun pada tahun 1904 dan selesai pada tahun 907 terletak di bundaran tugu muda yang dahulu disebut wilhelminaplein
lawang-sewu tempo doeloe ada kereta api lewat juga di depannya
di lantai dua terdapat aula besar di dulu digunakan sebagai ruang

# POSLDA

## HMM Tagger

In [7]:
# import
from hmmtagger.tagger import MainTagger
from tokenization import *

In [8]:
# initialize
tagger = MainTagger("resource/Lexicon.trn", "resource/Ngram.trn", 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)

In [9]:
# do process
doc_tagged = []

for tweet in doc_preprocessed:
    if len(tweet) == 0: continue
    out = sentence_extraction(cleaning(tweet))

    join_token = []
    for o in out:
        strtag = " ".join(tokenisasi_kalimat(o)).strip()
        join_token.extend(tagger.taggingStr(strtag))
    doc_tagged.append(' '.join(join_token))

In [10]:
doc_tagged

['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN di/IN bawah/NN lantai/NN dan/CC di/IN atap/IN itulah/DT fungsi/NN utama/JJ basement/NN',
 'lawang-sewu/NN berada/VBI di/IN ujung/VBT bodjongweg/NN atau/CC yang/SC sekarang/NN dikenal/VBT dengan/IN nama/NN jalan/NN pemuda/NN',
 'lawang-sewu/NN pernah/RB digunakan/VBT sebagai/IN penjara/NN bawah/NN tanah/NN oleh/IN serdadu/NN jepang/NN',
 'lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN yang/SC paling/RB dikenal/VBT para/DT turis/NN baik/JJ lokal/NN maupun/CC asing/JJ',
 'lawang-sewu/NN adalah/VBT salah/JJ satu/CDP bangunan/NN bersejarah/VBI yang/SC dibangun/VBT oleh/IN pemerintahan/NN kolonial/JJ belanda/NN pada/IN 27/CDP februari/NN 1904/CDP',
 'lawang-sewu/NN dulu/RB menjadi/VBT lokasi/NN pertempuran/NN yang/SC hebat/JJ antara/IN pemuda/NN angkatan/NN muda/JJ kereta/NN api/NN melawan/VBT kempetai/NN dan/CC kidobutai/NN jepang/NN',
 'masyarakat/NN setempat/NN menyebutnya/VBT lawang-sewu/NN seribu/NN pintu/NN dikar

## Kelas Tag

In [11]:
# define
Ccon = ['JJ', 'NN','NNP', 'NNG', 'VBI', 'VBT']
Cfnc = ['OP', 'CP', 'GM', ';', ':', '"', '.',
         ',', '-', '...', 'RB', 'IN', 'MD', 'CC',
         'SC', 'DT', 'UH', 'CDO', 'CDC', 'CDP', 'CDI',
         'PRP', 'WP', 'PRN', 'PRL', 'NEG', 'SYM', 'RP', 'FW']

In [12]:
# do process
doc_classified = []

for tweet in doc_tagged:
    tweet_split = tweet.split(' ')
    
    temp = {"Content": [], "Function": []}
    con = []
    fnc = []
    
    for token in tweet_split:
        word = token.split('/', 1)[0]
        tag = token.split('/', 1)[1]
        
        if tag in Ccon:
            con.append(token)
        elif tag in Cfnc:
            fnc.append(token)
            
    temp["Content"].append(' '.join(con))
    temp["Function"].append(' '.join(fnc))
    
    doc_classified.append(temp)

In [13]:
doc_classified

[{'Content': ['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN bawah/NN lantai/NN fungsi/NN utama/JJ basement/NN'],
  'Function': ['di/IN dan/CC di/IN atap/IN itulah/DT']},
 {'Content': ['lawang-sewu/NN berada/VBI ujung/VBT bodjongweg/NN sekarang/NN dikenal/VBT nama/NN jalan/NN pemuda/NN'],
  'Function': ['di/IN atau/CC yang/SC dengan/IN']},
 {'Content': ['lawang-sewu/NN digunakan/VBT penjara/NN bawah/NN tanah/NN serdadu/NN jepang/NN'],
  'Function': ['pernah/RB sebagai/IN oleh/IN']},
 {'Content': ['lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN dikenal/VBT turis/NN baik/JJ lokal/NN asing/JJ'],
  'Function': ['yang/SC paling/RB para/DT maupun/CC']},
 {'Content': ['lawang-sewu/NN adalah/VBT salah/JJ bangunan/NN bersejarah/VBI dibangun/VBT pemerintahan/NN kolonial/JJ belanda/NN februari/NN'],
  'Function': ['satu/CDP yang/SC oleh/IN pada/IN 27/CDP 1904/CDP']},
 {'Content': ['lawang-sewu/NN menjadi/VBT lokasi/NN pertempuran/NN hebat/JJ pemuda/NN angkatan/NN muda/JJ

In [14]:
# split document content and function
doc_content = []
for tweet in doc_classified:
    doc_content.append(''.join(tweet['Content']))

print(doc_content)
print()

# split tag and word
doc_prepared = []
for tweet in doc_content:
    tweet_split = tweet.split(' ')
    
    temp = []
    for token in tweet_split:
        word = token.split('/', 1)[0]
        temp.append(word)
    
    doc_prepared.append(temp)

print(doc_prepared)

['lawang-sewu/NN memiliki/VBT sistem/NN pendingin/NN air/NN bawah/NN lantai/NN fungsi/NN utama/JJ basement/NN', 'lawang-sewu/NN berada/VBI ujung/VBT bodjongweg/NN sekarang/NN dikenal/VBT nama/NN jalan/NN pemuda/NN', 'lawang-sewu/NN digunakan/VBT penjara/NN bawah/NN tanah/NN serdadu/NN jepang/NN', 'lawang-sewu/NN merupakan/VBT landmark/NN kota/NN semarang/NN dikenal/VBT turis/NN baik/JJ lokal/NN asing/JJ', 'lawang-sewu/NN adalah/VBT salah/JJ bangunan/NN bersejarah/VBI dibangun/VBT pemerintahan/NN kolonial/JJ belanda/NN februari/NN', 'lawang-sewu/NN menjadi/VBT lokasi/NN pertempuran/NN hebat/JJ pemuda/NN angkatan/NN muda/JJ kereta/NN api/NN melawan/VBT kempetai/NN kidobutai/NN jepang/NN', 'masyarakat/NN setempat/NN menyebutnya/VBT lawang-sewu/NN seribu/NN pintu/NN dikarenakan/VBT bangunan/NN memiliki/VBT pintu/NN banyak/JJ', 'lawang-sewu/NN dibangun/VBT tahun/NN selesai/NN tahun/NN terletak/VBI bundaran/NN tugu/NN muda/JJ dahulu/JJ disebut/VBT wilhelminaplein/NN', 'lawang-sewu/NN tempo/N

## LDA

In [15]:
# import
from lda.ldamodel import LdaModel

In [16]:
%%time
# iniatialize
k = 4
alpha = 0.01
beta = 0.01
iterations = 1000

# do process
lda = LdaModel(doc_prepared, k, alpha, beta, iterations)

80
Wall time: 5.54 s


In [17]:
result = lda.get_topic_word_pwz(doc_content)

df_lda = pd.DataFrame(result, columns=['Topik', 'Kata', 'PWZ'])

In [18]:
df_lda.head()

,Topik,Kata,PWZ
0,0,lawang-sewu/NN,0.086494
1,0,bangunan/NN,0.057759
2,0,dibangun/VBT,0.057759
3,0,pintu/NN,0.057759
4,0,tahun/NN,0.057759


In [19]:
print(df_lda[df_lda['Topik']==2]['PWZ'].to_string(index=False))

0.101007
0.067450
0.067450
0.067450
0.067450
0.033893
0.033893
0.033893
0.033893
0.033893
0.033893
0.033893
0.033893
0.033893
0.033893


In [20]:
lda.perplexity()

3.9722691941716524

# Grammar

In [21]:
from collections import defaultdict

dict_ldapwz = defaultdict(list)

In [22]:
for idx, row in df_lda.iterrows():
    dict_ldapwz[row['Topik']].append([row['Kata'], row['PWZ']])

In [23]:
dict_ldapwz

defaultdict(list,
            {0: [['lawang-sewu/NN', 0.08649425287356322],
              ['bangunan/NN', 0.05775862068965517],
              ['dibangun/VBT', 0.05775862068965517],
              ['pintu/NN', 0.05775862068965517],
              ['tahun/NN', 0.05775862068965517],
              ['muda/JJ', 0.05775862068965517],
              ['salah/JJ', 0.029022988505747128],
              ['belanda/NN', 0.029022988505747128],
              ['februari/NN', 0.029022988505747128],
              ['seribu/NN', 0.029022988505747128],
              ['memiliki/VBT', 0.029022988505747128],
              ['wilhelminaplein/NN', 0.029022988505747128],
              ['adalah/VBT', 0.029022988505747128],
              ['bersejarah/VBI', 0.029022988505747128],
              ['pemerintahan/NN', 0.029022988505747128]],
             1: [['lawang-sewu/NN', 0.07499999999999998],
              ['jepang/NN', 0.07499999999999998],
              ['kereta/NN', 0.07499999999999998],
              ['api/NN', 0.07

In [24]:
from grammar import CFG

In [29]:
cfg = CFG()

dict_story = cfg.create_sentences_from_data(dict(dict_ldapwz))

In [30]:
dict_story

{0: ['seribu yang salah muda',
  'tahun muda salah',
  'wilhelminaplein yang muda muda',
  'pintu muda terdapat bangunan',
  'pintu muda merupakan pintu muda',
  'bangunan yang muda bersejarah',
  'pemerintahan bersejarah'],
 1: ['api memiliki kempetai',
  'lawang-sewu melawan kereta di kempetai',
  'kidobutai terdapat kempetai',
  'kempetai terdapat doeloe',
  'lokasi sebagai lewat sebagai kempetai',
  'angkatan yaitu lawang-sewu',
  'jepang memiliki depannya'],
 2: ['aula mempunyai lantai dan lawang-sewu',
  'lawang-sewu besar besar',
  'kali yang utama sebagai pesta dan sistem',
  'air yang besar yaitu bawah',
  'sistem yang utama ada',
  'lantai yang besar yaitu bawah dan lawang-sewu',
  'bawah merupakan aula dan lawang-sewu'],
 3: ['bodjongweg yang asing baik',
  'jalan sebagai bodjongweg yang asing',
  'nama baik dikenal lokal',
  'jalan asing',
  'landmark yang asing merupakan turis',
  'lawang-sewu berada',
  'bodjongweg sebagai semarang dan bodjongweg']}